# Figshare API

In [1]:
#import libraries
import requests as rq
import pandas as pd
from pprint import pprint as pp
import pickle

API access tokens have been stored in credentials.pkl file

In [2]:
#import API access token
#at the moment, only have a single token in this credentials.pkl, which isn't the long term plan - need to change this
FIGSHARE_TOKEN = pickle.load(open('credentials.pkl', 'rb'))

In [ ]:
#Search seems to be all by type (articles, collections, etc.)
#All are public
#Plan is to search (1) articles, (2) collections, (3) projects, (4) data
#Article search may be minimally useful, but if have linked DOIs in object, could be linked data

#Can set up to cycle through which search (articles, collections, data) and by page

Overall workflow: use main search to get IDs of matching objects, then use IDs to get full object details & associated files

## Specify search parameters

In [3]:
#FIGSHARE_TOKEN imported from credentials.pkl
HEADERS = {'Authorization': 'token '+ FIGSHARE_TOKEN}

#Specify search terms
SEARCH = 'machine learning'

#Specify page to return with search (results are paginated)
PAGE = 1

#Specify number of results included on a page (default is 10, max is 1000)
PAGE_SIZE = 10

#Having variable definition outside PARAMS so can put into loop later

#Specify page and page size parameters
#Other search options are available, including limit and offset, but at the moment page/page size seem most useful
    #if set both page/page size and limit/offset, get:
    #{'message': 'Pagination options can be set either via page/page_size or limit/offset params','code': 'ConflictingPaginationOptions'}
    
    #'limit': 1000, #Number of results included on a page. Used for pagination with query (optional) - not sure how differs from page_size
    #'offset': 1 #, #Where to start the listing(the offset of the first result). Used for pagination with limit (optional)
#We don't need to specify other search parameters like institution, group, modified since, etc.

#Specify which search (collections, articles, projects, data)
ARTICLE_URL = 'https://api.figshare.com/v2/articles'
COLLECTIONS_URL = "https://api.figshare.com/v2/collections/search"
PROJECTS_URL = "https://api.figshare.com/v2/projects/search"
#DATA_URL = ???

#Full search parameters
PARAMS = {
    'search_for': SEARCH, #search term
    'page': PAGE, 
    'page_size': PAGE_SIZE,  
    }

In [ ]:
## Iterate through pages until get response: {'message': 'Bad Request', 'code': 'BadRequest'}
## Seems like there should be a better way

## Example: search public articles

In [4]:
## Run search for public articles
response = rq.get(ARTICLE_URL, params=PARAMS, headers=HEADERS)

In [5]:
response

<Response [200]>

In [6]:
## Put output into json format
output = response.json()

In [7]:
## See what output looks like
output

[{'id': 6714890,
  'title': 'Attribute Learning using Joint Human and Machine Computation',
  'doi': '10.1184/r1/6714890.v1',
  'handle': '',
  'url': 'https://api.figshare.com/v2/articles/6714890',
  'published_date': '2012-08-01T00:00:00Z',
  'thumb': 'https://s3-eu-west-1.amazonaws.com/ppreviews-cmu-49810598590254/12247874/thumb.png',
  'defined_type': 8,
  'defined_type_name': 'thesis',
  'group_id': 18614,
  'url_private_api': 'https://api.figshare.com/v2/account/articles/6714890',
  'url_public_api': 'https://api.figshare.com/v2/articles/6714890',
  'url_private_html': 'https://figshare.com/account/articles/6714890',
  'url_public_html': 'https://kilthub.cmu.edu/articles/thesis/Attribute_Learning_using_Joint_Human_and_Machine_Computation/6714890',
  'timeline': {'posted': '2012-08-01T00:00:00',
   'firstOnline': '2018-06-30T23:16:04',
   'revision': '2018-06-30T23:16:05'},
  'resource_title': '',
  'resource_doi': ''},
 {'id': 6716609,
  'title': 'Expressive Collaborative Music P

In [8]:
#Convert output to pd dataframe and see table format
df = pd.DataFrame(output)

df

,id,title,doi,handle,url,published_date,thumb,defined_type,defined_type_name,group_id,url_private_api,url_public_api,url_private_html,url_public_html,timeline,resource_title,resource_doi
0,6714890,Attribute Learning using Joint Human and Machi...,10.1184/r1/6714890.v1,,https://api.figshare.com/v2/articles/6714890,2012-08-01T00:00:00Z,https://s3-eu-west-1.amazonaws.com/ppreviews-c...,8,thesis,18614,https://api.figshare.com/v2/account/articles/6...,https://api.figshare.com/v2/articles/6714890,https://figshare.com/account/articles/6714890,https://kilthub.cmu.edu/articles/thesis/Attrib...,"{'posted': '2012-08-01T00:00:00', 'firstOnline...",,
1,6716609,Expressive Collaborative Music Performance via...,10.1184/r1/6716609.v1,,https://api.figshare.com/v2/articles/6716609,2016-08-01T00:00:00Z,https://s3-eu-west-1.amazonaws.com/ppreviews-c...,8,thesis,18614,https://api.figshare.com/v2/account/articles/6...,https://api.figshare.com/v2/articles/6716609,https://figshare.com/account/articles/6716609,https://kilthub.cmu.edu/articles/thesis/Expres...,"{'posted': '2016-08-01T00:00:00', 'firstOnline...",,
2,6720413,"Learning with Sparcity: Structures, Optimizati...",10.1184/r1/6720413.v1,,https://api.figshare.com/v2/articles/6720413,2013-07-01T00:00:00Z,https://s3-eu-west-1.amazonaws.com/ppreviews-c...,8,thesis,18614,https://api.figshare.com/v2/account/articles/6...,https://api.figshare.com/v2/articles/6720413,https://figshare.com/account/articles/6720413,https://kilthub.cmu.edu/articles/thesis/Learni...,"{'posted': '2013-07-01T00:00:00', 'firstOnline...",,
3,6720416,Learning with Staleness,10.1184/r1/6720416.v1,,https://api.figshare.com/v2/articles/6720416,2018-03-01T00:00:00Z,https://s3-eu-west-1.amazonaws.com/ppreviews-c...,8,thesis,18614,https://api.figshare.com/v2/account/articles/6...,https://api.figshare.com/v2/articles/6720416,https://figshare.com/account/articles/6720416,https://kilthub.cmu.edu/articles/thesis/Learni...,"{'posted': '2018-03-01T00:00:00', 'firstOnline...",,
4,6720908,On Learning from Collective Data,10.1184/r1/6720908.v1,,https://api.figshare.com/v2/articles/6720908,2013-12-01T00:00:00Z,https://s3-eu-west-1.amazonaws.com/ppreviews-c...,8,thesis,18614,https://api.figshare.com/v2/account/articles/6...,https://api.figshare.com/v2/articles/6720908,https://figshare.com/account/articles/6720908,https://kilthub.cmu.edu/articles/thesis/On_Lea...,"{'posted': '2013-12-01T00:00:00', 'firstOnline...",,
5,6720836,New Paradigms and Optimality Guarantees in Sta...,10.1184/r1/6720836.v1,,https://api.figshare.com/v2/articles/6720836,2017-12-01T00:00:00Z,https://s3-eu-west-1.amazonaws.com/ppreviews-c...,8,thesis,18614,https://api.figshare.com/v2/account/articles/6...,https://api.figshare.com/v2/articles/6720836,https://figshare.com/account/articles/6720836,https://kilthub.cmu.edu/articles/thesis/New_Pa...,"{'posted': '2017-12-01T00:00:00', 'firstOnline...",,
6,6720878,Nonparametric Learning in High Dimensions,10.1184/r1/6720878.v1,,https://api.figshare.com/v2/articles/6720878,2010-12-01T00:00:00Z,https://s3-eu-west-1.amazonaws.com/ppreviews-c...,8,thesis,18614,https://api.figshare.com/v2/account/articles/6...,https://api.figshare.com/v2/articles/6720878,https://figshare.com/account/articles/6720878,https://kilthub.cmu.edu/articles/thesis/Nonpar...,"{'posted': '2010-12-01T00:00:00', 'firstOnline...",,
7,6720410,Learning with Limited Supervision by Input and...,10.1184/r1/6720410.v1,,https://api.figshare.com/v2/articles/6720410,2012-05-01T00:00:00Z,https://s3-eu-west-1.amazonaws.com/ppreviews-c...,8,thesis,18614,https://api.figshare.com/v2/account/articles/6...,https://api.figshare.com/v2/articles/6720410,https://figshare.com/account/articles/6720410,https://kilthub.cmu.edu/articles/thesis/Learni...,"{'posted': '2012-05-01T00:00:00', 'firstOnline...",,
8,6724586,Why Machine Learning Works,10.1184/r1/6724586.v1,,https://api.figshare.com/v2/articles/6724586,2017-12-01T00:00:00Z,https://s3-eu-west-1.amazonaws.com/ppreviews-c...,8,thesis,18614

In [ ]:
## Alternative approach - normalize output
## Same output as pd dataframe - thinking stick with pd for ease of data wrangling downstream
#df2 = pd.json_normalize(output)
#df2

In [9]:
## Extract IDs
article_ids = list(df.id)

In [14]:
## View article details

#URL syntax is: https://api.figshare.com/v2/articles/{article_id}

#create empty pandas dataframe
df_article_details = pd.DataFrame()

#for each ID, 
for i in article_ids:
    id_i = str(i)
    URL_i = 'https://api.figshare.com/v2/articles/' + id_i
    #print(URL_i)
    response_i = rq.get(URL_i, headers=HEADERS)
    json_i = response_i.json()
    #json_normalize collapses first level, which is a start
    #for now, can leave files, custom fields, author, etc as list of dictionary
    df_i = pd.json_normalize(json_i)
    df_article_details = df_article_details.append(df_i)

6714890
https://api.figshare.com/v2/articles/6714890
6716609
https://api.figshare.com/v2/articles/6716609
6720413
https://api.figshare.com/v2/articles/6720413
6720416
https://api.figshare.com/v2/articles/6720416
6720908
https://api.figshare.com/v2/articles/6720908
6720836
https://api.figshare.com/v2/articles/6720836
6720878
https://api.figshare.com/v2/articles/6720878
6720410
https://api.figshare.com/v2/articles/6720410
6724586
https://api.figshare.com/v2/articles/6724586
14402201
https://api.figshare.com/v2/articles/14402201


In [15]:
df_article_details

,files,custom_fields,authors,figshare_url,description,funding,funding_list,version,status,size,...,url_public_html,resource_title,resource_doi,license.value,license.name,license.url,timeline.posted,timeline.firstOnline,timeline.revision,timeline.submission
0,"[{'id': 12247874, 'name': 'Attribute Learning ...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5364275, 'full_name': 'Edith L.M. Law'...",https://kilthub.cmu.edu/articles/thesis/Attrib...,<p>This thesis is centered around the problem ...,None,[],1,public,8863551,...,https://kilthub.cmu.edu/articles/thesis/Attrib...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2012-08-01T00:00:00,2018-06-30T23:16:04,2018-06-30T23:16:05,NaN
0,"[{'id': 12249995, 'name': 'Expressive Collabor...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5464304, 'full_name': 'Guangyu Xia', ...",https://kilthub.cmu.edu/articles/thesis/Expres...,<p>Techniques of Artificial Intelligence and H...,None,[],1,public,11070988,...,https://kilthub.cmu.edu/articles/thesis/Expres...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2016-08-01T00:00:00,2018-07-01T00:04:06,2018-07-01T00:04:07,NaN
0,"[{'id': 12254777, 'name': 'Learning with Sparc...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5362001, 'full_name': 'Xi Chen', 'is_...",https://kilthub.cmu.edu/articles/thesis/Learni...,<p>The development of modern information techn...,None,[],1,public,3773851,...,https://kilthub.cmu.edu/articles/thesis/Learni...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2013-07-01T00:00:00,2018-07-01T00:25:42,2018-07-01T00:25:43,NaN
0,"[{'id': 12254780, 'name': 'Learning with Stale...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 4947985, 'full_name': 'Wei Dai', 'is_a...",https://kilthub.cmu.edu/articles/thesis/Learni...,<p>A fundamental assumption behind most machin...,None,[],1,public,4664211,...,https://kilthub.cmu.edu/articles/thesis/Learni...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2018-03-01T00:00:00,2018-07-01T00:25:52,2018-07-01T00:25:53,NaN
0,"[{'id': 12255338, 'name': 'On Learning from Co...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5363849, 'full_name': 'Liang Xiong', ...",https://kilthub.cmu.edu/articles/thesis/On_Lea...,In many machine learning problems and applicat...,None,[],1,public,4152506,...,https://kilthub.cmu.edu/articles/thesis/On_Lea...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2013-12-01T00:00:00,2018-07-01T00:48:03,2018-07-01T00:48:04,NaN
0,"[{'id': 12255260, 'name': 'New Paradigms and O...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5466287, 'full_name': 'Yu-Xiang Wang'...",https://kilthub.cmu.edu/articles/thesis/New_Pa...,Machine learning (ML) has become one of the mo...,None,[],1,public,8304735,...,https://kilthub.cmu.edu/articles/thesis/New_Pa...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2017-12-01T00:00:00,2018-07-01T00:44:53,2018-07-01T00:44:54,NaN
0,"[{'id': 12255305, 'name': 'Nonparametric Learn...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5362709, 'full_name': 'Han Liu', 'is_...",https://kilthub.cmu.edu/articles/thesis/Nonpar...,This thesis develops flexible and principled n...,None,[],1,public,4835357,...,https://kilthub.cmu.edu/articles/thesis/Nonpar...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2010-12-01T00:00:00,2018-07-01T00:46:48,2018-07-01T00:46:50,NaN
0,"[{'id': 12254774, 'name': 'Learning with Limit...","[{'name': 'Publisher Statement', 'value': ''},...","[{'id': 5360768, 'full_name': 'Yi Zhang', 'is...",https://kilthub.cmu.edu/articles/thesis/Learni...,<p>In many real-world applications of supervis...,None,[],1,public,1611768,...,https://kilthub.cmu.edu/articles/thesis/Learni...,None,None,43,In Copyright,http://rightsstatements.org/vocab/InC/1.0/,2012-05-01T00:00:00,2018-07-01T00:25:35,2018-07-01T0

In [ ]:
## Combine with higher level metadata

#although...if details includes everything from original call, no need to combine - just keep detailed metadata output

In [ ]:
## List files associated with public articles (by ID)

#URL syntax is: https://api.figshare.com/v2/articles/{article_id}/files

#looks like files are already pulled from article details, so don't need a separate call